1 - Qual o ticket médio das vendas por idade ordenado por idade?

2 - Qual o total de vendas por idade ordenado pelo valor total de vendas?

3 - Liste a quantidade dos 10 produtos mais vendidos 
 
4 - Quantos usuários únicos existem na base?

5 - Liste os 10 usuários com o maior valor total de vendas

6 - Qual porcentagem representa cada faixa etária da base de usuários únicos?

## Carregando as bibliotecas

In [21]:
import pandas as pd
import os
import json
from pathlib import Path

In [ ]:
# Criando o caminho relativo da pasta com os arquivos JSON e capturando o nome dos arquivos
data_dir_caminho = Path('..') / 'data'
arquivos_json = [arq for arq in os.listdir(data_dir_caminho) if arq.endswith('.json')]

In [ ]:
# Carrega e concatena (parâmetro lines=True para evitar o aviso de Warning - lê cada linha como um objeto JSON separado)
dfs = []
for arquivo in arquivos_json:
    caminho = os.path.join(data_dir_caminho, arquivo)
    df = pd.read_json(caminho, lines=True)
    dfs.append(df)

df_final = pd.concat(dfs, ignore_index=True)
print(f"DataFrame final: {len(df_final)} linhas")

DataFrame final: 550068 linhas


próximos passos:

- fazer a análise exploratória

- responder as perguntas de negócio